In [1]:
# Import dependencies
import pandas as pd
from pathlib import Path

In [2]:
# Read in data from csv file
wine_data = Path("Resources/winemag-data-130k-v2.csv")


In [3]:
# Convert to dataframe
wine_df = pd.read_csv(wine_data)
wine_df.head()

,id,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [4]:
wine_df.columns

Index(['id', 'country', 'description', 'designation', 'points', 'price',
       'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

In [9]:
# Clean and transform data
# Drop null values
# Drop unnecessary columns
reduced_wine_df = wine_df.loc[:,["country", "description", "points",
                                    "region_1", "variety"]]
reduced_wine_df

,country,description,points,region_1,variety
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,Etna,White Blend
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,NaN,Portuguese Red
2,US,"Tart and snappy, the flavors of lime flesh and...",87,Willamette Valley,Pinot Gris
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,Lake Michigan Shore,Riesling
4,US,"Much like the regular bottling from 2012, this...",87,Willamette Valley,Pinot Noir
...,...,...,...,...,...
129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,90,NaN,Riesling
129967,US,Citation is given as much as a decade of bottl...,90,Oregon,Pinot Noir
129968,France,Well-drained gravel soil gives this wine its c...,90,Alsace,Gewürztraminer
129969,France,"A dry style of Pinot Gris, this is crisp with ...",90,Alsace,Pinot Gris


In [12]:
reduced_wine_df = reduced_wine_df.dropna(how='any')
reduced_wine_df

,country,description,points,region_1,variety
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,Etna,White Blend
2,US,"Tart and snappy, the flavors of lime flesh and...",87,Willamette Valley,Pinot Gris
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,Lake Michigan Shore,Riesling
4,US,"Much like the regular bottling from 2012, this...",87,Willamette Valley,Pinot Noir
5,Spain,Blackberry and raspberry aromas show a typical...,87,Navarra,Tempranillo-Merlot
...,...,...,...,...,...
129965,France,"While it's rich, this beautiful dry wine also ...",90,Alsace,Pinot Gris
129967,US,Citation is given as much as a decade of bottl...,90,Oregon,Pinot Noir
129968,France,Well-drained gravel soil gives this wine its c...,90,Alsace,Gewürztraminer
129969,France,"A dry style of Pinot Gris, this is crisp with ...",90,Alsace,Pinot Gris


In [ ]:
# Transform words into vectors using NLP
    # Tokenize descriptions to split sentences into word components
    # Drop stop words
    # Lemmitization
# Separate data in target and features variables
# Preprocess data
# Divide into train and test data

In [ ]:
# Input vector data into ML algorithm
# Apply K-Means Clustering
# Generate scatter plot of results

In [ ]:
# Input vector data into ML algorithm
# Apply 2nd Model (TBD)
# Generate scatter plot of results

In [ ]:
# Classify using Random Forest

In [ ]:
# Print accuracy score and confusion matrix